# Hypothesis Testing

Before we talk about the uses, procedures, or computational methods behind Hypothesis testing I think it is important to address that this topic is mired in some level of misinformation and misconception<sup>[1](https://www.tandfonline.com/doi/full/10.1080/10691898.2009.11889514)</sup>. Misconceptions in this subject appear at many levels within academia even amongst faculty, so it is understandable why this is generally such a hard subject for most folks to wrap their heads around. 

This notebook is a humble attempt to:

1. Address some of the most common missconceptions around hyposthesis testing.
2. Give some basic literacy around statistical concepts important to hypothesis testing.
3. Illustrate how to make sense of hypthosis testing and use it in a real world context.


<sup>1</sup> [How Confident are Students in their Misconceptions about Hypothesis Tests?](https://www.tandfonline.com/doi/full/10.1080/10691898.2009.11889514)

### What is Hypothesis Testing?

Hypothesis testing refers to a method of statistical ***inference*** in which (*usually*) we compare two statistical data sets. 
Data alone is often uninteresting, it is the ***interpretation*** of data that is really of value in almost all contexts <sup>[2](https://machinelearningmastery.com/statistical-hypothesis-tests/)</sup>.

I believe *Jason Brownlee, PHD* puts it best:
>"In statistics, when we wish to start asking questions about the data and interpret the results, we use statistical methods that provide a confidence or likelihood about the answers. In general, this class of methods is called statistical hypothesis testing, or significance tests.  A hypothesis test calculates some quantity under a given assumption. The result of the test allows us to interpret whether the assumption holds or whether the assumption has been violated."

#### Things hypothesis test methods may be used to evaluate
- Does a data sample have a Gaussian distribution? (E.g. Shapiro-Wilk Test)
- Do two samples have a linear relationship (E.g. Peason's Correlation Coefficient)
- Do two samples have a [monotonic relationship](https://www.statisticshowto.com/monotonic-relationship/). (E.g. Kendall's Rank Correlation)
- Are two categorical variables related or independant? (E.g. Chi-Squared Test)
- Are the means of two independant samples signficantly different? (E.g. Student's T-Test)

This is a non-exhustive list but it is meant to illustrate some of the common questions that are asked as part of a hypothesis test and the method that is used to make that evaluation. It is import to remember that these tests are ***inferential*** and always come with some in-built assumptions, so your interpretation should always account for those assumptions.

<sup>2</sup> [A Gentle Introduction into Statistical Hypothesis Testing](https://machinelearningmastery.com/statistical-hypothesis-tests/)

#### Hypothesis Testing Fomulation

A hypothesis test is always formulated so that the assumptions of a given test is the default.

The default or **Null Hypothesis** is denoted as *H<sub>0</sub>*

While the **Alternative Hypothesis** is denoted as *H<sub>1</sub>*

*Example (Student's T-Test)*

*H<sub>0</sub>*: The means of the samples are equal

*H<sub>1</sub>*: The means of the samples are unequal

Once we have interpreted the results of the test (which commonly take the form of either a *p-value* or *critical value*) we can than start to make claims and either reject the *H<sub>0</sub>* or fail to reject the *H<sub>0</sub>*.


###  Hypothesis Testing Misconceptions

1. Hypothesis Tests are not mathematical (logical) proofs of one of the two Hypotheses
    - Results of a hypothesis test are not deterministic and therefore cannot prove a Null(*H<sub>0</sub>*) or Alternative (*H<sub>1</sub>*) hypothese to be true or false with 100% certainty.


2. Hypothesis Tests do not act as a probabilistic proof by contradiction
    - Rejecting a Null (*H<sub>0</sub>*) Hypothesis based upon a test is not irrefutable evidence that the Alternative (*H<sub>1</sub>*) Hypothesis is true and vice-versa.

In [ ]:
# new modification 2